In [ ]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
import ipdb
import pickle

### read all timesurfaces and associated labels from the first layer

In [ ]:
%%capture
with open('layer_data.pkl', 'rb') as file:
    first = pickle.load(file)
surfs = first.all_timesurfaces.copy()
# delete all empty surfaces
#surfs = surfs[surfs.sum(axis=(1,2,3)) != 0]
# reshape into 2D array (flattening surfaces)
#surfs = surfs.reshape(-1,11*11)

labels = first.network.all_labels.copy()

### Cluster all timesurfaces

In [ ]:
n_of_centers = 16
kmeans = KMeans(n_clusters=n_of_centers).fit(surfs)
centers = kmeans.cluster_centers_.reshape(-1,11,11)

fig, axes = plt.subplots(4, 4)
axes = np.reshape(axes, -1)

for index, axis in enumerate(axes):
    axis.imshow(centers[index], vmin=0, vmax=1,
                cmap = plt.cm.hot, interpolation = 'none', origin = 'upper')

### Cut the array of timesurfaces and labels along recording borders and make histograms for each recording

In [ ]:
print('Original labels for events')
print(labels[2730:2740])
print(labels.shape)
print('Kmeans labels')
print(kmeans.labels_[2730:2740])
print(kmeans.labels_.shape)

recording_indices = np.where(labels[:-1] != labels[1:])[0]
recording_indices += 1
print('Recording indices: ' + str(len(recording_indices)))
print(recording_indices)

kmeans_split = np.split(kmeans.labels_, recording_indices)
label_split = np.split(labels, recording_indices)

hists = []
[hists.append(np.histogram(x, bins=np.arange(0,n_of_centers+1))[0]) for x in kmeans_split]
short_labels = []
[short_labels.append(x[0]) for x in label_split]

print(short_labels)

import collections
collections.Counter(short_labels)

In [ ]:
# split X and y into training and testing sets
from sklearn.cross_validation import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=0)

In [ ]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
# fit the model with data
logreg.fit(X_train,y_train)
y_pred=logreg.predict(X_test)

In [ ]:
first.network
